In [1]:
import kfp
from kfp.components import create_component_from_func

In [10]:
INGRESS_GATEWAY='http://istio-ingressgateway.istio-system.svc.cluster.local'
EXPERIMENT="DemoModelServing"
NAMESPACE="dev-profile"

In [11]:
COOKIE="authservice_session="+'MTY0NjM1NTA5N3xOd3dBTkVwQlJqVk5Oak5aUTBGSFFWUTJSVXcxVTBKSldWQkJURnBHVDBkYU5sTk5SMVpXTWtoWlVqWlFUVlkyTlVaYVdFVlJOa0U9fOB7TAUsImQ3WllMpjUYyhNKE9NlNFTEbT7G3lEu-6ae'
client = kfp.Client(host=INGRESS_GATEWAY+"/pipeline", cookies=COOKIE)

In [12]:
import kfp
from kfp import dsl
import json
import os
from kfp.onprem import use_k8s_secret
from kfp import components
from kfp.components import load_component_from_file, load_component_from_url
from kfp import dsl
from kfp import compiler
from kfp.components import create_component_from_func


def deploy_isvc(service_name, namespace, storage_uri):
    import kfp
    resource = """
apiVersion: serving.kserve.io/v1beta1
kind: InferenceService
metadata:
  name: {}
  namespace: {}
  annotations:
    sidecar.istio.io/inject: "false"
spec:
  predictor:
    serviceAccountName: minio-sa
    containers:
      - name: kserve-container
        image: pytorch/torchserve:latest-cpu
        args:
          - torchserve
          - --start
          - --no-config-snapshots
          - --model-store=/mnt/models/model-store
          - --ts-config=/mnt/models/config/config.properties
          - --models all
        env:
          - name: STORAGE_URI
            value: {}
""".format(service_name, namespace, storage_uri)
    rop = dsl.ResourceOp(
        name="create-inference-service",
        k8s_resource=resource,
        action="create",
        attribute_outputs={"name": "{.metadata.name}"}
    )
    
    dsl.get_pipeline_conf().add_op_transformer(
        use_k8s_secret(
            secret_name="minio-secret",
            k8s_secret_key_to_env={
                "secretkey": "MINIO_SECRET_KEY",
                "accesskey": "MINIO_ACCESS_KEY",
            },
        )
    )

deploy_isvc_op = create_component_from_func(
    deploy_isvc, output_component_file='isvc_component.yaml')

@dsl.pipeline(
    name="Deploy a model in pipeline", description="foobar"
)
def deploy_pytorch_model(service_name, namespace, storage_uri):
    deploy_isvc_op(service_name, namespace, storage_uri)

In [13]:
arguments = {'service_name': 'demo-isvc-kfp', 'namespace': 'dev-profile', 'storage_uri': 's3://mlpipeline/models/pytorch'}

# Create a pipeline run, using the client you initialized in a prior step.
client.create_run_from_pipeline_func(deploy_pytorch_model, arguments=arguments)

RunPipelineResult(run_id=99b6a48d-1534-4c78-809b-ffcdd40c36b0)